In [259]:
import pandas as pd
import numpy as np

In [260]:
class Validator:
    def __init__(self, method, df):
        self.df = df
        self.method = method
        markers_method_columns = [
            ('lat1', float),('lon1', float),
            ('popup1',str),('marker_size','int64')
        ]
        route_method_columns = [
            ('lat1',float),('lon1',float),
            ('lat2',float),('lon2',float),
            ('popup1',str),('popup2',str),
            ('line_width','int64'),('line_color',str),
        ]
        markers_method_df = pd.DataFrame(
            {k: pd.Series(dtype=t) for k, t in markers_method_columns}
        )
        route_method_df = pd.DataFrame(
            {k: pd.Series(dtype=t) for k, t in route_method_columns}
        )
        self.methods_dfs = {
            'markers': markers_method_df,
            'route': route_method_df
        }
        if method in ['markers','heatmap']:
            self.coordinates_columns_count = 2
        elif method in ['route','graph']:
            self.coordinates_columns_count = 4
        else:
            raise TypeError
        self.method_columns = self.methods_dfs[method].columns
        
    def columns_validator(self):
        our_df_columns = set(self.df.columns)
        method_df_columns = set(self.method_columns)
        #issubset не подходит, если в фрейме меньше нужных колонок но остальные подходят
        return our_df_columns & method_df_columns == method_df_columns
    
    def column_type_validator(self):
        #our_df-создает словарь имя_колонки:тип_данных для входного фрейма
        our_df = dict(self.df[self.method_columns].dtypes)
        # method_df-создает словарь имя_колонки:тип_данных для фрейма методов
        method_df = dict(self.methods_dfs[self.method].dtypes)
        return our_df == method_df
    
    def column_nan_validator(self):
        return  all(self.df.replace('', np.nan).notna().all())
    
    def coordinates_validator(self):
        # ЕЩЕ РАЗ ПРОВЕРИТЬ ИНТЕРВАЛЫ ДЛЯ ДОЛГОТЫ И ШИРОТЫ
        def latlng_validator(latlngs, type_coord):
            #если значение широта,оно должно быть в интервале [-90,90]
            if type_coord == 'lat':
                left_limit, right_limit = -90, 90
            #если значение долгота,оно должно быть в интервале [-180,180]
            elif type_coord =='lon':
                left_limit, right_limit = -180, 180
            latlng_validation = []
            for latlng in latlngs:
                # с помощью lambda-выражения проверяем элементы списка на принадлежность заданному интервалу
                validation = list(
                    map(
                        lambda x: True if x >= left_limit and x <= right_limit else False,
                        latlng
                    )
                )
                latlng_validation.append(all(validation))
            # если все значения внутри списка True, вернет True
            return all(latlng_validation)
        if self.coordinates_columns_count == 2:
            result_validation = [
                latlng_validator(
                    [self.df['lat1'].tolist()],'lat'
                ),
                latlng_validator(
                    [self.df['lon1'].tolist()], 'lon'
                )
            ]
        else:
            result_validation = [
                latlng_validator([
                    self.df['lat1'].tolist(),
                    self.df['lat2'].tolist()
                ],'lat'),
                latlng_validator([
                    self.df['lon1'].tolist(),
                    self.df['lon2'].tolist()
                ],'lon')
            ]
        print(result_validation)
        return all(result_validation)
    

In [267]:
test1 = pd.DataFrame()
test1['lat1'] = [30.4567, 15.36788, 23.98123]
test1['lon1'] = [45.1234, 78.2345, 12.45656]
test1['lat2'] = [34.78989, 45.67888, 89.24325]
test1['lon2'] = [22.7765, 77.32456, 41.364568]
test1['popup1'] = ['text11', None, 'text13']
test1['popup2'] = ['text21', 'text22', 'text23']
test1['line_width'] = [3,5,7]
test1['line_color'] = ['red','black','blue']
test1['marker_size'] = [3,5,7]

In [233]:
test2 = pd.DataFrame()
test2['lat1'] = [30.4567, 50.36788, 23.98123]
test2['lon1'] = [45.1234, 78.2345, 12.45656]
test2['lat2'] = [34.78989, 45.67888, 89.24325]
test2['lon2'] = [22.7765, 77.32456, 41.364568]
test2['popup1'] = ['text11', 'text12', 'text13']
test2['popup2'] = ['text21', 'text22', 'text23']
test2['line_width'] = [3,5,7]
test2['line_color'] = ['red','black','blue']
test2['marker_size'] = [3,5,6]

In [268]:
v1 = Validator('route', test1)

In [269]:
v1.columns_validator()

True

In [270]:
v1.column_type_validator()

True

In [271]:
v1.column_nan_validator()

False

In [272]:
v1.coordinates_validator()

[True, True]


True